<a href="https://colab.research.google.com/github/drew-walkerr/Diss_Detecting_Provider_Bias/blob/main/stigmatizing_labels_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score




import torch
torch.cuda.empty_cache()
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)
from google.colab import output
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

#GPU usage setup
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.9 MB/s eta 0:00:00


In [ ]:
# mount drive to access data
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:

%cd /content/drive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Stigmatizing Labels/3_Classification

%ls

/content/drive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Stigmatizing Labels/3_Classification
annotation_Drew_sample_stigmatizing_labels.csv
gold_standard_stigma_100.csv
predictions.csv
stigmatizing_labels_accuracy.gsheet
stigmatizing_labels_classification.ipynb


In [ ]:
df = pd.read_csv("gold_standard_stigma_100.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   sentence_id   100 non-null    int64 
 1   sentence      100 non-null    object
 2   matched_term  100 non-null    object
 3   stigma_label  100 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 3.2+ KB


In [ ]:

df['label'] = df['stigma_label']
df.drop(['sentence_id','stigma_label'], axis=1)
del df['sentence_id'], df['stigma_label']
df.head(3)

,sentence,matched_term,label
0,USED SNEAKERS ONCE AND BOOTS ONCE DURING NIGHT...,\brefusing\b,1
1,nsg progress note 7a-7p neuro: At start of shi...,\bhostile\b,1
2,"Plan: Pt will need feeding tube placement, rem...",\bdelusional\b,0


In [ ]:
cols = ['sentence', 'matched_term']
df['text'] = df[cols].apply(lambda row: '</s>'.join(row.values.astype(str)), axis=1)
df.head(3)

,sentence,matched_term,label,text
0,USED SNEAKERS ONCE AND BOOTS ONCE DURING NIGHT...,\brefusing\b,1,USED SNEAKERS ONCE AND BOOTS ONCE DURING NIGHT...
1,nsg progress note 7a-7p neuro: At start of shi...,\bhostile\b,1,nsg progress note 7a-7p neuro: At start of shi...
2,"Plan: Pt will need feeding tube placement, rem...",\bdelusional\b,0,"Plan: Pt will need feeding tube placement, rem..."


In [ ]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 10
LEARNING_RATE = 5e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

In [ ]:
class BiasData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        #print(self.targets.shape)

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
newdf = df[['text','label']]
newdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    100 non-null    object
 1   label   100 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [ ]:
train_size = 0.8
train_data=newdf.sample(frac=train_size,random_state=0)
test_data=newdf.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(newdf.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = BiasData(train_data, tokenizer, MAX_LEN)
testing_set = BiasData(test_data, tokenizer, MAX_LEN)

FULL Dataset: (100, 2)
TRAIN Dataset: (80, 2)
TEST Dataset: (20, 2)


In [ ]:
train_data['label'].unique()

array([1, 0])

In [ ]:
test_data['label'].unique()

array([1, 0])

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.0)
        self.classifier = torch.nn.Linear(768, 2) #output labels = 1 for binary classification

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = RobertaClass()
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print(f"Training Loss: {loss_step}")
            print(f"Training Accuracy: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss: 0.675177812576294
Training Accuracy: 62.5


10it [00:09,  1.01it/s]


The Total Accuracy for Epoch 0: 55.0
Training Loss Epoch: 0.6932971477508545
Training Accuracy Epoch: 55.0


0it [00:00, ?it/s]

...

The Total Accuracy for Epoch 8: 96.25
Training Loss Epoch: 0.16895803313236685
Training Accuracy Epoch: 96.25


0it [00:00, ?it/s]

Training Loss: 1.0912623405456543
Training Accuracy: 75.0


10it [00:07,  1.39it/s]

The Total Accuracy for Epoch 9: 86.25
Training Loss Epoch: 0.47560832407325504
Training Accuracy Epoch: 86.25


In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0
    tr_loss=0
    nb_tr_steps=0
    nb_tr_examples=0

    all_preds = []  # list to store predictions
    all_targets = []  # list to store original targets
    all_texts = []  # list to store original input texts

    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)

            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)

            all_preds.extend(big_idx.cpu().numpy())  # store predictions
            all_targets.extend(targets.cpu().numpy())  # store targets

            all_texts.extend(data['ids'])  # store original input texts

            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples += targets.size(0)

            if _ % 5000 == 0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss: {loss_step}")
                print(f"Validation Accuracy: {accu_step}")

    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples

    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    # Print classification report
    report = classification_report(all_targets, all_preds)
    print(report)

    # Confusion matrix
    cm = confusion_matrix(all_targets, all_preds)
    print("Confusion Matrix:")
    print(cm)

    # Create a DataFrame and save it
    df_predictions = pd.DataFrame({
        'Text': all_texts,
        'Original': all_targets,
        'Predicted': all_preds
    })
    df_predictions.to_csv('predictions.csv', index=False)
    print(df_predictions.head())

    return epoch_accu


Initial Run -- one interation

In [ ]:
valid(model, testing_loader)

acc = valid(model,testing_loader)
print("test accuracy = %0.2f%%" % acc)


1it [00:00,  4.41it/s]

Validation Loss: 1.0407683849334717
Validation Accuracy: 37.5


3it [00:00,  5.06it/s]


Validation Loss Epoch: 1.246919830640157
Validation Accuracy Epoch: 45.0
              precision    recall  f1-score   support

           0       0.20      0.12      0.15         8
           1       0.53      0.67      0.59        12

    accuracy                           0.45        20
   macro avg       0.37      0.40      0.37        20
weighted avg       0.40      0.45      0.42        20

Confusion Matrix:
[[1 7]
 [4 8]]
                                                Text  Original  Predicted
0  [tensor(0), tensor(13043), tensor(10928), tens...         0          1
1  [tensor(0), tensor(2515), tensor(2653), tensor...         1          0
2  [tensor(0), tensor(45935), tensor(3048), tenso...         0          1
3  [tensor(0), tensor(565), tensor(5073), tensor(...         1          0
4  [tensor(0), tensor(21243), tensor(29758), tens...         1          1


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
1it [00:00,  4.43it/s]

Validation Loss: 1.1318169832229614
Validation Accuracy: 37.5


3it [00:00,  5.14it/s]


Validation Loss Epoch: 0.9312431315581003
Validation Accuracy Epoch: 45.0
              precision    recall  f1-score   support

           0       0.20      0.12      0.15         8
           1       0.53      0.67      0.59        12

    accuracy                           0.45        20
   macro avg       0.37      0.40      0.37        20
weighted avg       0.40      0.45      0.42        20

Confusion Matrix:
[[1 7]
 [4 8]]
                                                Text  Original  Predicted
0  [tensor(0), tensor(565), tensor(5073), tensor(...         1          0
1  [tensor(0), tensor(13043), tensor(10928), tens...         0          1
2  [tensor(0), tensor(45935), tensor(3048), tenso...         0          1
3  [tensor(0), tensor(31842), tensor(9764), tenso...         1          1
4  [tensor(0), tensor(1090), tensor(3275), tensor...         0          0
test accuracy = 45.00%


In order to account for variations in GPU states, we ran each of the models 10 times and provide the mean accuracy


## Predictions

In [ ]:
def predict_on_dataframe(df, model, tokenizer, max_len):
    model.eval()
    predictions = []

    for index, row in df.iterrows():
        text = row['text']
        inputs = tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = torch.tensor([inputs['input_ids']], dtype=torch.long).to(device)
        mask = torch.tensor([inputs['attention_mask']], dtype=torch.long).to(device)
        token_type_ids = torch.tensor([inputs["token_type_ids"]], dtype=torch.long).to(device)

        with torch.no_grad():
            outputs = model(ids, mask, token_type_ids)

        big_val, big_idx = torch.max(outputs.data, dim=1)
        predictions.append(big_idx[0].item())

    df['predictions'] = predictions
    return df


In [ ]:
predicted_classes = predict_on_dataframe(newdf, model, tokenizer, max_len=128)
print(predicted_classes)

annotation_drew_sample = pd.read_csv("annotation_Drew_sample_stigmatizing_labels.csv")
annotation_drew_sample.info()
annotation_drew_sample= annotation_drew_sample.rename(columns={'Sentence':'text'})

predicted_mega = predict_on_dataframe(annotation_drew_sample, model, tokenizer, max_len=128)
predicted_mega
#
filtered_df = predicted_mega[predicted_mega['predictions'] == 1]

filtered_df

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
<ipython-input-26-bbb1cbe3f7d1>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predictions'] = predictions


                                                 text  label  predictions
0   USED SNEAKERS ONCE AND BOOTS ONCE DURING NIGHT...      1            1
1   nsg progress note 7a-7p neuro: At start of shi...      1            1
2   Plan: Pt will need feeding tube placement, rem...      0            0
3   Essentially stable though presenting with stra...      1            1
4   Pt at times refuses to reposition and CDB.</s>...      1            1
..                                                ...    ...          ...
95      Refusing po's s/p extubation.</s>\brefusing\b      0            0
96  sepsis) p/w ischemic colitis; s/p ex-lap, R co...      0            0
97  Infant demanding feeds, awake and alert for fe...      0            0
98  Refuses to wear humidified O2 and pneumo boots...      0            0
99  Spiriva with HandiHaler *NF* 18 mcg Inhalation...      1            1

[100 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (to

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


,Unnamed: 0.1,Unnamed: 0,Sentence ID,ROW_ID_x,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,...,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,text,regex,matched_term,stigmatizing_patient_label,annotator_comments,predictions
3,75027,17329787,17329787,1422148,12170,158721.0,2191-06-05,2191-06-05 05:22:00,2191-06-05 05:26:00,Nursing/other,...,NaN,NaN,NaN,0,"Can be dangerously agitated, attempting to get...","chat_gpt|\ ""hysterical|\ aggressive|\ drug\ ad...",\ dangerous,NaN,NaN,1
5,24000,2532384,2532384,1977702,21768,160497.0,2127-06-17,2127-06-17 13:10:00,2127-06-17 13:15:00,Nursing/other,...,NaN,NaN,NaN,0,Taking 50-60cc q. 4 hrs without difficulty.,"chat_gpt|\ ""hysterical|\ aggressive|\ drug\ ad...",\ difficult,NaN,NaN,1
6,26406,3522215,3522215,2033766,26098,143258.0,2143-09-14,2143-09-14 18:16:00,2143-09-14 18:32:00,Nursing/other,...,NaN,NaN,NaN,0,"spells, still with some breathing regulation d...","chat_gpt|\ ""hysterical|\ aggressive|\ drug\ ad...",\ difficult,NaN,NaN,1
8,35752,7296019,7296019,53182,30138,199890.0,2122-01-19,NaN,NaN,Discharge summary,...,NaN,NaN,NaN,0,She has difficulty describing what her seizure...,"chat_gpt|\ ""hysterical|\ aggressive|\ drug\ ad...",\ difficult,NaN,NaN,1
10,43797,10438215,10438215,1481516,16775,140347.0,2187-11-27,2187-11-27 05:37:00,2187-11-27 06:08:00,Nursing/other,...,2188-04-23,2188-04-23,2188-04-23,1,Neuro: On bipap mask all night so difficult f...,"chat_gpt|\ ""hysterical|\ aggressive|\ drug\ ad...",\ difficult,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,36651,7584814,7584814,34352,30084,162715.0,2197-06-14,NaN,NaN,Discharge summary,...,NaN,NaN,NaN,0,Discharge Instructions: You were admitted to ...,"chat_gpt|\ ""hysterical|\ aggressive|\ drug\ ad...",\ difficult,NaN,NaN,1
492,71874,10558990,10558990,399048,21090,185350.0,2120-01-24,2120-01-24 10:51:00,2120-01-24 10:51:45,Physician,...,2120-02-02,2120-02-02,2120-02-02,1,HPI: 24 Hour Events: NON-INVASIVE VENTILATION...,"chat_gpt|\ ""hysterical|\ aggressive|\ drug\ ad...",\ unstable,NaN,NaN,1
493,25003,2946166,2946166,688187,68629,128041.0,2147-07-16,2147-07-16 04:46:00,2147-07-16 07:25:31,Physician,...,NaN,NaN,NaN,0,TSICU HPI: 75M w SAH / intraparenchymal bleed ...,"chat_gpt|\ ""hysterical|\ aggressive|\ drug\ ad...",\ difficult,NaN,NaN,1
497,67718,7002688,7002688,694062,54348,123562.0,2164-10-03,2164-10-03 14:12:00,2164-10-03 14:31:08,Nursing,...,NaN,NaN,NaN,0,Wean to PSV Response: Secretions have decreas...,"chat_gpt|\ ""hysterical|\ aggressive|\ drug\ ad...",\ dramatic,NaN,NaN,1


In [ ]:

# constant for classes
#classes = ('Neither Source/Location', 'Source, no user', 'User, no source', 'User and source')


#print(test_data["label"])
#predicted_roberta = model.
#predict(test_data["text"])
#print(predicted)
# Build confusion matrix
#cf_matrix = confusion_matrix(test_data["label"], predicted)
#df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
#                     columns = [i for i in classes])
#plt.figure(figsize = (4,4))
#sn.heatmap(df_cm, annot=True)
#plt.savefig('output.png')

Notes for next time-- want to turn this script into a function where I can pass through the hyperparameters.

In [ ]:
# svm baseline

#from sklearn.pipeline import make_pipeline
#from sklearn.preprocessing import StandardScaler
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.svm import SVC

#from sklearn.pipeline import Pipeline
#text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', SVC(gamma='auto'))])
#text_clf.fit(train_data['text'], train_data['label'])

In [ ]:
#docs_test = test_data['text']
#predicted = text_clf.predict(docs_test)
#np.mean(predicted == test_data['label'])

In [ ]:
#test_accs = []
#for i in range(10):
#  model = RobertaClass()
#  model.to(device)
#
#  loss_function = torch.nn.CrossEntropyLoss()
#  optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
#
#  for epoch in range(EPOCHS):
#      train(epoch)
#  acc = valid(model, testing_loader)
#  #print("test accuracy = %0.2f%%" % acc)
#  test_accs.append(acc)

# mean_acc = sum(test_accs) / len(test_accs)
# print("MEAN ACCURACY = %0.2f%%" % mean_acc)
# output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/d/d9/Wilhelm_Scream.ogg").play()')
